In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import tsdm
from tsdm.datasets import TimeSeriesCollection
from tsdm.random.samplers import HierarchicalSampler, SlidingWindowSampler

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
ds = tsdm.datasets.KIWI_RUNS()

In [ ]:
ds.timeseries

In [ ]:
ds.timeseries

In [ ]:
ds.time_features.dtypes

In [ ]:
from tsdm.datasets import KIWI

In [ ]:
ds = KIWI()
ds

In [ ]:
ds[439, 15325]

In [ ]:
from tsdm.tasks import KIWI_RUNS_TASK

task = KIWI_RUNS_TASK()

In [ ]:
TSC = TimeSeriesCollection(
    index=ds.index,
    timeseries=ds.timeseries,
    metadata=ds.metadata,
    time_features=ds.time_features,
    value_features=ds.value_features,
    metadata_features=ds.metadata_features,
)

## Sampler

In [ ]:
ts = ds.timeseries.loc[(439, 15325)]

In [ ]:
sampler = SlidingWindowSampler(ts.index, horizons=["2h", "1h"], stride="1h")

In [ ]:
next(iter(sampler))

## Construct the Sampler

In [ ]:
from tsdm.datasets import *
from tsdm.tasks import *

In [ ]:
task = KiwiTask()
task.splits[0, "train"]

In [ ]:
task.train_partition

In [ ]:
task.dataloaders[0, "train"]

In [ ]:
task = InSilicoTask()
task.validate_folds()

In [ ]:
mask = task.folds[0, "train"]
mask

In [ ]:
task.dataset.timeseries.loc[mask[mask].index]

In [ ]:
df

In [ ]:
idx = self.dataset.index

df = idx.to_frame(index=False).set_index(idx.names)

groups = df.groupby(idx.names, sort=False).ngroup()

In [ ]:
groups

In [ ]:
groups.name

In [ ]:
fold

In [ ]:
fold = task.folds[0, "train"]
# task.dataset[fold]

In [ ]:
task.splits[0, "train"]

In [ ]:
task.index

In [ ]:
task.splits[0, "train"]

In [ ]:
KiwiTask().folds

In [ ]:
TimeSeriesCollection(InSilicoData().dataset)[16130]

In [ ]:
from torch.utils.data import SubsetRandomSampler

In [ ]:
from tsdm.tasks import KiwiTask

In [ ]:
task = KiwiTask()

In [ ]:
next(iter(SubsetRandomSampler(TSC.index)))

In [ ]:
TSC[(439, 15325)]

In [ ]:
subsamplers = {
    key: SlidingWindowSampler(ds.timeseries.index, horizons=["2h", "1h"], stride="1h")
    for key, ds in TSC.items()
}
sampler = HierarchicalSampler(TSC, subsamplers, shuffle=False)

In [ ]:
from tsdm.datasets import InSilicoData

ds = InSilicoData()

ds.timeseries

In [ ]:
outer_key, (forecasting_horizon, prediction_horizon) = next(iter(sampler))

In [ ]:
TSC[outer_key][forecasting_horizon]

# TimeSeriesCollectionForecastingTask

In [ ]:
TSC

In [ ]:
from tsdm.tasks import TimeSeriesTaskDataset

targets = ["Base", "DOT", "Glucose", "OD600"]
observables = [
    "Base",
    "DOT",
    "Glucose",
    "OD600",
    "Acetate",
    "Fluo_GFP",
    "pH",
]
covariates = [
    "Cumulated_feed_volume_glucose",
    "Cumulated_feed_volume_medium",
    "InducerConcentration",
    "StirringSpeed",
    "Flow_Air",
    "Temperature",
    "Probe_Volume",
]

In [ ]:
key = next(iter(sampler))
outer_key, (observation_horizon, forecasting_horizon) = key
TSC[outer_key][observation_horizon]

In [ ]:
TSC[439, 15325]

# Collction Test

In [ ]:
task = TimeSeriesTaskDataset(
    TSC,
    targets=targets,
    observables=observables,
    covariates=covariates,
    sample_format=("sparse", "sparse"),
)
sample = task[key]

# Dataset Test

In [ ]:
task = TimeSeriesTaskDataset(
    TSC[439, 15325],
    targets=targets,
    observables=observables,
    covariates=covariates,
    sample_format=("masked", "masked"),
)
sample = task[key[1]]

In [ ]:
raise

In [ ]:
if sample.inputs.t_target is not None:
    diff = sample.inputs.t_target.index.difference(sample.targets.y.index)
    sample.inputs.t_target.drop(diff, inplace=True)
sample

In [ ]:
raise

In [ ]:
diff = sample.inputs.t_target.index.difference(sample.targets.y.index)
sample.inputs.t_target.drop(diff, inplace=True)
sample

In [ ]:
raise

# Mapping Dataset

In [ ]:
d = dict(enumerate("asdfghjkl"))

In [ ]:
from collections.abc import Mapping
from dataclasses import dataclass

from torch.utils.data import Dataset

In [ ]:
@dataclass
class MyMapping(Dataset, Mapping):
    internal_dict: dict

    def __iter__(self):
        return iter(self.internal_dict)

    def __getitem__(self, key):
        return self.internal_dict[key]

    def __len__(self):
        return len(self.internal_dict)

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(MyMapping(d))

In [ ]:
from typing import Any, Callable, Sequence


def flatten_dict(
    d: dict[str, Any],
    /,
    *,
    recursive: bool = True,
    join_fn: Callable[[Sequence[str]], str] = ".".join,
) -> dict[str, Any]:
    r"""Flatten dictionaries recursively."""
    result = {}
    for key, item in d.items():
        if isinstance(item, dict) and recursive:
            subdict = flatten_dict(item, recursive=True, join_fn=join_fn)
            for subkey, subitem in subdict.items():
                result[join_fn((key, subkey))] = subitem
        else:
            result[key] = item
    return result


def unflatten_dict(
    d: dict[str, Any],
    /,
    *,
    recursive: bool = True,
    split_fn: Callable[[str], Sequence[str]] = lambda s: s.split(".", maxsplit=1),
) -> dict[str, Any]:
    r"""Unflatten dictionaries recursively."""
    result = {}
    for key, item in d.items():
        split = split_fn(key)
        result.setdefault(split[0], {})
        if len(split) > 1 and recursive:
            assert len(split) == 2
            subdict = unflatten_dict(
                {split[1]: item}, recursive=recursive, split_fn=split_fn
            )
            result[split[0]] |= subdict
        else:
            result[split[0]] = item
    return result


a = {
    "a": True,
    "b": 42,
    "c": "foo",
    "foo": {"a": 1, "b": 2, "c": 3},
    "bar": {"a": {"a": 1}, "b": {"a": 1}, "c": {"a": 1}},
    "baz": {"a": True, "b": 2},  # <-- does not work!
}

print(flat := flatten_dict(a))
print(unflat := unflatten_dict(a))
assert unflat == a

In [ ]:
flat

In [ ]:
unflatten_dict(flat)

In [ ]:
"a".split(".")

In [ ]:
dir(".")